## Device use T4 * 2

#### Clone WMamba & U-Mamba Repo

In [ ]:
!git clone https://github.com/Yui-Arthur/WMamba.git
%cd WMamba
!git clone https://github.com/bowang-lab/U-Mamba.git

#### Mabma & U-Mamba & WMamba reqirement

In [ ]:
!pip install causal-conv1d>=1.2.0 --no-cache-dir > /dev/null
!pip install mamba-ssm --no-cache-dir > /dev/null
!cd U-Mamba/umamba && (pip install -e .) > /dev/null

In [ ]:
!pip install ptwt >  /dev/null
!pip install PyWavelets >  /dev/null

In [ ]:
!sudo bash setup_wmamba.sh

#### set env variable

In [ ]:
### don't edit ###
%env nnUNet_preprocessed=/home/nnUNet_preprocessed/
%env nnUNet_raw=/home/nnUNet_raw/
%env nnUNet_results=/kaggle/working/

# set training epochs
%env training_epochs=1

### dataset prepaired

In [ ]:
"""===== Dataset 110 Covid-19 ===="""
""" unrar data folder and move to home. """
# !apt install unrar
# !(unrar x /kaggle/input/covid-19/Dataset110_CTImage.mp4 /home/nnUNet_raw/) >  /dev/null
"""dataset preoproccess about 20 mins."""
# !nnUNetv2_plan_and_preprocess -d 110 --verify_dataset_integrity -c 3d_fullres
# dataset_id = 110
"""===== Dastaset 4 Hippocampus ===="""
""" unzip data folder and move to home. """
# !(tar -C  /home/ -xvf /kaggle/input/covid-19/Task04_Hippocampus.mp4) > /dev/null
""" dataset preoproccess. """
# !nnUNetv2_convert_MSD_dataset -i /home/Task04_Hippocampus
# !nnUNetv2_plan_and_preprocess -d 4 --verify_dataset_integrity -c 3d_fullres
# dataset_id = 4

### training

In [ ]:
!CUDA_VISIBLE_DEVICES=0 nnUNetv2_train {dataset_id} 3d_fullres 0 -tr nnUNetTrainerWMambaBot & \
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train {dataset_id} 3d_fullres 1 -tr nnUNetTrainerWMambaBot 

Pre-train

In [ ]:
checkpoint_path = "/kaggle/input/covid19-checkpoint/checkpoint_final.pth"

In [ ]:
!CUDA_VISIBLE_DEVICES=0 nnUNetv2_train {dataset_id} 3d_fullres 0 -tr nnUNetTrainerWMambaBot -pretrained_weights  {checkpoint_path} & \
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train {dataset_id} 3d_fullres 1 -tr nnUNetTrainerWMambaBot -pretrained_weights  {checkpoint_path}

In [ ]:
"""===== Dastaset 4 Hippocampus ===="""
!nnUNetv2_find_best_configuration {dataset_id} -c 3d_fullres -tr nnUNetTrainerWMambaBot -f 0 1 

In [ ]:
!nnUNetv2_predict -i "/home/nnUNet_raw/Dataset110_CTImage/imagesTs" -o "/kaggle/working/Dataset110_CTImage/predTs0" -d {dataset_id} -c 3d_fullres -f 0 -tr nnUNetTrainerWMambaBot 

In [ ]:
!nnUNetv2_predict -i "/home/nnUNet_raw/Dataset110_CTImage/imagesTs" -o "/kaggle/working/Dataset110_CTImage/predTs1" -d {dataset_id} -c 3d_fullres -f 1 -tr nnUNetTrainerWMambaBot